# Adversarial Search: Solving Tic-Tac-Toe with Monte Carlo Tree Search

## Introduction 

Multiplayer games can be implemented as:
1. Nondeterministic actions: The opponent is seen as part of an environment with nondeterministic actions. Non-determinism is the result of the unknown opponent's moves. 
2. Optimal Decisions: Minimax search (search complete game tree) and alpha-beta pruning.
3. Heuristic Alpha-Beta Tree Search: Cut off tree search and use heuristic to estimate state value. 
4. __Monte Carlo Search:__ Simulate playouts to estimate state value. 

Here we will implement search for Tic-Tac-Toe (see [rules](https://en.wikipedia.org/wiki/Tic-tac-toe)). The game is a __zero-sum game__: Win by x results in +1, win by o in -1 and a tie has a value of 0. Max plays x and tries to maximize the outcome while Min plays o and tries to minimize the outcome.   

We will implement
* Pure Monte Carlo search

## The board

I represent the board as a vector of length 9. The values are `' ', 'x', 'o'`.  

In [2]:
import numpy as np
import math

In [3]:
def empty_board():
    return [' '] * 9

board = empty_board()
display(board)

[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']

Some helper functions.

In [4]:
def show_board(board):
    """display the board"""
    b = np.array(board).reshape((3,3))
    print(b)

board = empty_board()
show_board(board)    

print()
print("Add some x's")
board[0] = 'x'; board[3] = 'x'; board[6] = 'x';  
show_board(board)

[[' ' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]

Add some x's
[['x' ' ' ' ']
 ['x' ' ' ' ']
 ['x' ' ' ' ']]


In [5]:
def check_win(board):
    """check the board and return one of x, o, d (draw), or n (for next move)"""
    
    board = np.array(board).reshape((3,3))
    
    diagonals = np.array([[board[i][i] for i in range(len(board))], 
                          [board[i][len(board)-i-1] for i in range(len(board))]])
    
    for a_board in [board, np.transpose(board), diagonals]:
        for row in a_board:
            if len(set(row)) == 1 and row[0] != ' ':
                return row[0]
    
    # check for draw
    if(np.sum(board == ' ') < 1):
        return 'd'
    
    return 'n'

show_board(board)
print('Win? ' + check_win(board))

print()
show_board(empty_board())
print('Win? ' + check_win(empty_board()))

[['x' ' ' ' ']
 ['x' ' ' ' ']
 ['x' ' ' ' ']]
Win? x

[[' ' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]
Win? n


In [6]:
def get_actions(board):
    """return possible actions as a vector ot indices"""
    return np.where(np.array(board) == ' ')[0].tolist()

    # randomize the action order
    #actions = np.where(np.array(board) == ' ')[0]
    #np.random.shuffle(actions)
    #return actions.tolist()


show_board(board)
get_actions(board)

[['x' ' ' ' ']
 ['x' ' ' ' ']
 ['x' ' ' ' ']]


[1, 2, 4, 5, 7, 8]

In [7]:
def result(state, player, action):
    """Add move to the board."""
    
    state = state.copy()
    state[action] = player
  
    return state

show_board(empty_board())

print()
print("State for placing an x at position 4:")
show_board(result(empty_board(), 'x', 4))

[[' ' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]

State for placing an x at position 4:
[[' ' ' ' ' ']
 [' ' 'x' ' ']
 [' ' ' ' ' ']]


In [8]:
def other(player): 
    if player == 'x': return 'o'
    else: return 'x'

In [9]:
def utility(state, player = 'x'):
    """check is a state is terminal and return the utility if it is. None means not a terminal mode."""
    goal = check_win(state)        
    if goal == player: return +1 
    if goal == 'd': return 0  
    if goal == other(player): return -1  # loss is failure
    return None # continue

print(utility(['x'] * 9))
print(utility(['o'] * 9))
print(utility(empty_board()))

1
-1
None


# Pure Monte Carlo Search

See AIMA page 161ff. 

We implement a extremely simplified version.

For the current state: 
1. Simulate $N$ random playouts for each possible action and 
2. pick the action with the highest average utility.

__Important note:__ we use here a random playout policy, which ends up creating just a randomized search that works fine for this toy problem. For real applications you need to extend the code with
1. a good __playout policy__ (e.g., learned by self-play) and 
2. a __selection policy__ (e.g., UCB1).

## Simulate playouts

In [13]:
def playout(state, action, player = 'x'):
    """Perfrom a random playout starting with the given action on the fiven board 
    and return the utility of the finished game."""
    state = result(state, player, action)
    current_player = other(player)
    
    while(True):
        # reached terminal state?
        u = utility(state, player)
        if u is not None: return(u)
        
        # we use a random playout policy
        a = np.random.choice(get_actions(state))
        print(a)
        state = result(state, current_player, a)
        #print(state)
        
        # switch between players
        current_player = other(current_player)


board = empty_board()
print(playout(board, 0))

4
2
3
8
5
-1


In [11]:
def playouts(board, action, player = 'x', N = 100):
    """Perform N playouts following the given action for the given board."""
    return [ playout(board, action, player) for i in range(N) ]

p = playouts(board, 0)
print(p)

print(f"mean utility: {np.mean(p)}")
print(f"win probability: {sum(np.array(p) == +1)/len(p)}")
print(f"loss probability: {sum(np.array(p) == -1)/len(p)}")

3
8
2
1
5
4
1
8
2
4
1
3
6
7
4
2
8
5
7
8
6
1
5
3
2
4
5
1
6
4
7
8
2
4
5
1
7
6
8
8
2
4
3
1
7
6
5
6
3
7
1
8
6
4
2
3
1
5
2
5
8
4
3
7
1
6
5
4
1
2
3
8
2
8
6
7
5
4
3
4
8
6
5
1
2
3
1
4
2
4
7
3
8
1
2
6
5
6
8
2
3
1
7
5
4
8
2
4
5
1
7
3
6
4
2
6
7
3
8
5
7
3
2
6
2
8
5
4
4
6
5
2
8
3
6
5
2
4
8
7
1
3
6
3
7
5
8
2
8
3
1
7
4
3
5
8
7
6
4
1
2
1
8
7
5
4
7
2
6
3
8
8
1
3
2
7
3
2
6
4
2
3
7
1
6
5
6
4
2
7
3
5
1
8
2
8
5
1
6
4
1
4
5
2
3
8
1
2
8
3
6
4
7
1
6
4
2
5
8
7
6
2
3
4
7
1
4
1
3
7
2
8
5
7
8
4
5
6
1
2
3
8
6
7
5
4
6
3
8
5
2
4
8
5
2
6
7
4
1
3
8
5
4
3
2
7
6
6
7
8
2
5
4
1
3
2
8
7
1
3
6
5
4
8
3
2
4
6
1
7
8
4
1
7
5
3
2
7
3
1
5
4
1
2
5
8
3
4
5
1
4
3
8
7
6
2
3
4
8
1
7
2
2
5
4
8
7
3
1
3
4
8
7
1
2
5
6
6
8
7
2
4
3
1
4
8
6
5
7
2
7
1
2
5
6
8
4
3
1
7
4
5
8
2
8
1
6
7
4
4
6
7
5
1
4
2
5
7
6
3
1
8
6
3
7
1
8
1
4
3
5
8
7
6
2
6
7
2
8
1
5
4
4
5
6
3
7
2
8
8
5
2
6
3
1
7
4
1
7
3
8
4
6
4
2
3
1
2
3
6
5
8
7
1
4
4
6
8
2
3
5
7
1
1
2
5
3
6
8
7
4
7
2
3
1
5
8
2
6
1
4
5
7
3
8
2
6
5
2
6
8
7
1
7
8
4
2
3
5
5
7
8
1
2
4
7
6
1
2
7
8
4
5
6
2
1
5
2
3
7


__Note:__ This shows that the player who goes first has a significant advantage in __pure random play.__ A better playout policy would be good.

## Pure Monte Carlo Search

In [12]:
DEBUG = 1

def pmcs(board, N = 100, player = 'x'):
    """Pure Monte Carlo Search. Returns the action that has the largest average utility.
    The N playouts are evenly divided between the possible actions."""
    global DEBUG
    
    actions = get_actions(board)
    n = math.floor(N/len(actions))
    if DEBUG >= 1: print(f"Actions: {actions} ({n} playouts per actions)")
    
    ps = { i:np.mean(playouts(board, i, player, N = n)) for i in actions }
    if DEBUG >= 1: display(ps)
        
    action = max(ps, key=ps.get)
    return action

board = empty_board()
display(board)
%timeit -r1 -n1 print(pmcs(board))

print()
print("10000 playouts give a better utility estimate.")
%timeit -r1 -n1 print(pmcs(board, N = 10000))

[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']

Actions: [0, 1, 2, 3, 4, 5, 6, 7, 8] (11 playouts per actions)
1
5
2
4
3
7
8
6
8
7
5
4
6
1
4
8
7
5
1
3
6
5
1
7
4
8
2
8
6
7
3
6
2
1
7
8
3
5
4
5
8
6
7
3
2
4
1
6
2
8
7
5
3
4
8
6
7
1
5
2
7
2
1
4
5
6
4
1
3
7
6
5
2
3
4
7
6
2
0
8
5
7
4
8
3
5
2
6
0
3
7
5
4
2
6
8
4
7
5
8
0
2
6
3
5
4
3
0
2
7
2
6
0
5
3
8
7
4
8
6
2
7
4
3
0
7
8
3
6
4
0
5
2
3
6
4
0
5
2
4
7
0
3
8
8
4
6
2
7
5
0
6
3
4
7
8
1
1
3
0
5
7
4
1
3
4
5
7
1
4
0
3
5
6
8
5
0
6
4
8
0
1
6
5
3
1
0
8
4
3
5
7
6
8
4
0
7
1
6
4
3
8
6
7
1
5
0
7
5
3
4
0
8
3
5
4
0
6
8
8
2
4
6
1
5
0
0
2
4
7
6
8
1
5
7
1
2
6
8
4
0
5
7
1
0
2
6
5
8
0
4
8
5
7
0
4
1
8
6
1
6
7
2
4
4
6
7
2
0
1
8
8
6
0
2
4
1
8
0
2
7
5
1
6
4
8
5
0
3
6
1
5
0
2
2
5
3
6
1
7
0
7
0
8
3
1
6
3
7
1
0
6
8
6
1
8
5
3
0
7
0
8
3
2
1
5
2
6
8
7
0
1
6
1
0
2
5
8
3
0
8
6
7
1
5
2
6
5
3
1
7
0
2
8
2
1
6
0
7
3
5
8
0
6
2
3
7
8
1
3
0
7
6
8
4
2
1
3
2
8
6
1
0
4
7
7
0
4
6
2
1
3
8
8
4
6
1
0
2
7
7
8
0
1
6
2
3
0
6
8
7
2
8
1
4
7
2
6
3
0
3
1
2
4
8
0
6
7
7
6
3
0
4
2
1
0
7
8
2
3
1
4
8
7
2
0
3
4
1
5
7
4
8
0
3
2
3
0
1
4
2
8
2
8
3
7
8
5
4

{0: 0.2727272727272727,
 1: 0.0,
 2: 0.6363636363636364,
 3: 0.09090909090909091,
 4: 0.2727272727272727,
 5: -0.09090909090909091,
 6: 0.8181818181818182,
 7: 0.2727272727272727,
 8: -0.09090909090909091}

6
496 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

10000 playouts give a better utility estimate.
Actions: [0, 1, 2, 3, 4, 5, 6, 7, 8] (1111 playouts per actions)
3
8
5
2
6
1
1
3
8
4
7
5
1
3
8
7
6
2
4
5
4
2
6
1
6
7
1
3
8
5
2
4
7
8
3
1
5
4
5
2
6
8
4
3
7
1
7
2
3
5
4
6
1
5
3
8
1
6
2
1
3
8
5
2
4
6
2
8
1
1
8
6
3
5
4
4
6
8
7
5
1
2
1
7
4
3
5
6
3
1
6
4
2
8
6
7
8
4
5
3
1
2
2
5
6
1
4
3
7
4
8
6
1
2
5
7
1
4
3
2
6
8
4
3
8
2
6
7
1
5
3
7
1
2
6
5
8
4
5
3
2
6
7
6
8
3
4
3
6
7
5
1
2
2
1
7
8
5
6
4
3
2
7
4
1
5
3
6
1
5
2
8
6
7
4
6
2
7
1
8
5
4
1
6
2
5
4
3
2
6
8
6
4
2
3
7
1
5
8
1
6
8
2
7
4
8
6
2
7
3
4
1
5
5
3
4
1
2
8
6
3
1
7
4
8
5
2
6
7
8
5
2
3
4
8
3
1
4
7
2
5
6
2
3
8
7
6
4
1
5
6
4
3
5
8
2
1
7
6
1
8
2
4
3
1
7
6
2
8
5
6
2
1
5
7
8
6
1
2
5
3
4
8
7
4
3
2
5
8
7
1
6
1
3
4
8
5
6
3
8
2
4
2
4
7
5
1
6
3
8
3
2
4
8
6
1
7
1
3
5
2
8
4
6
5
3
2
6
4
3
8
2
1
7
5
6
2
8
4
7
3
5
1
6
7
4
5
8
5
1
7
3
6
4
2
8
4
2
5
7
8
3
6
1
8
1
7
6
5
2
1
5
6
8
7
4
2
6
8
5
4
7
1
3
6
7
1
3
2
4
8
5
5
8
3
2
6
4
2
5
7
1
4
8
6
3
7
6
2
5
4

4
1
7
3
5
1
7
4
8
6
2
5
3
1
4
8
6
5
3
2
7
1
6
1
8
4
2
6
7
5
3
3
6
5
1
2
7
8
5
3
2
7
6
4
1
8
2
8
6
5
4
6
1
3
2
3
1
5
7
6
8
2
4
5
6
1
3
6
8
7
1
2
3
4
4
8
6
2
1
7
3
5
2
8
3
4
5
3
2
4
7
6
4
5
2
8
1
6
7
3
5
1
2
8
7
4
6
3
5
8
1
4
2
8
5
7
3
6
8
4
2
5
3
6
7
1
5
6
8
1
7
3
2
3
7
4
6
5
6
5
2
3
4
6
1
5
3
4
8
7
2
4
8
7
5
3
6
1
5
2
8
4
1
7
3
6
3
2
7
5
4
6
8
1
1
8
4
6
2
5
3
7
3
2
6
4
5
8
3
1
8
2
4
1
7
5
6
3
2
1
5
8
3
2
7
6
4
6
3
1
5
8
7
2
4
6
1
3
2
5
2
3
6
1
8
7
4
3
6
2
1
4
7
8
5
8
2
5
4
3
6
6
8
5
3
7
2
4
1
4
3
5
1
2
8
6
4
7
2
5
8
6
1
3
4
2
6
7
1
8
3
5
1
4
3
7
8
6
5
2
8
2
4
7
1
3
5
6
6
7
4
2
3
5
1
8
7
4
8
1
5
6
3
2
8
7
4
5
1
6
2
3
7
1
8
6
4
5
2
3
2
8
1
6
3
5
7
4
6
1
3
4
2
8
8
4
6
3
2
7
5
7
1
8
4
5
6
3
2
3
7
2
8
4
1
6
6
8
4
5
1
7
2
8
1
2
7
5
8
5
3
6
7
1
4
2
2
5
4
1
7
3
8
6
3
2
7
8
1
4
6
8
4
5
3
1
7
2
1
5
4
6
3
7
8
2
4
7
6
8
1
3
5
2
8
1
7
4
3
6
2
5
6
2
3
4
8
5
1
7
7
5
4
6
8
2
3
1
6
2
4
8
1
5
1
4
3
5
8
6
2
7
6
2
4
8
3
7
5
1
3
4
8
2
5
7
7
4
3
5
1
8
2
5
4
3
1
8
6
8
2
3
6
5
4
2
3
7
4
5
1
8
5
1
7
4
6
3
8
2


...

1
0
7
3
4
2
4
3
8
1
7
4
5
8
2
0
5
1
2
3
4
0
4
5
2
3
7
0
8
0
1
7
5
2
3
4
3
8
2
1
0
7
5
1
8
0
4
3
1
2
8
4
8
5
7
3
1
0
5
8
4
7
7
2
1
4
0
5
4
3
1
8
2
4
2
3
8
5
3
8
0
2
7
4
5
7
1
0
2
3
7
4
1
2
7
5
1
8
4
4
1
5
0
8
3
3
0
7
8
5
2
4
0
8
3
2
4
5
1
4
7
0
2
3
0
5
4
7
3
2
8
4
3
8
2
1
7
0
0
8
4
2
5
7
2
0
1
5
4
7
3
8
3
0
5
7
2
8
0
4
8
7
2
3
1
3
8
1
2
7
4
0
4
5
2
5
3
7
1
8
2
4
0
0
5
8
1
4
7
8
4
5
1
3
4
1
7
0
2
1
7
0
2
5
4
8
3
1
4
7
5
4
0
7
1
8
2
7
5
4
0
2
1
3
8
8
4
0
2
0
2
1
8
5
3
7
4
5
8
7
3
1
2
4
3
8
1
7
2
8
3
0
4
7
7
1
0
4
5
2
0
1
4
8
5
2
7
3
4
1
3
2
0
7
8
2
4
0
3
1
2
4
8
5
0
3
1
8
3
2
5
7
3
1
5
2
7
4
5
2
1
4
0
2
8
4
3
2
7
5
4
0
1
3
8
1
4
2
0
1
2
5
0
7
8
3
4
1
2
0
4
2
7
8
3
4
0
7
8
0
5
2
4
1
3
8
7
4
2
0
3
2
5
1
7
4
5
1
2
4
7
8
0
3
1
3
0
7
4
2
8
4
3
1
0
1
8
7
2
5
4
1
3
8
5
0
7
2
0
4
8
2
2
4
7
0
1
8
4
8
3
7
2
5
7
4
1
3
4
0
7
2
8
5
1
0
7
1
5
3
8
1
4
3
2
0
2
3
4
7
4
2
0
3
8
1
3
5
4
0
8
2
5
8
3
7
1
2
7
3
0
4
4
3
7
5
8
2
1
2
8
0
1
3
7
8
0
5
1
7
3
2
1
3
7
4
5
0
8
4
8
2
7
1
3
0
5
8
7
4
3
1
5
8
7
2
4
4
5
8


8
1
5
4
1
8
6
3
2
0
1
6
3
0
2
4
8
5
1
6
2
0
4
3
1
8
3
4
5
6
4
1
6
0
5
8
3
8
5
6
2
1
4
3
0
1
2
8
0
4
3
6
5
8
2
5
6
3
1
4
0
8
1
2
3
6
8
6
4
0
1
2
5
3
0
3
1
4
8
2
6
5
2
4
0
8
1
6
8
2
1
5
0
3
4
2
5
8
1
4
0
6
0
3
2
4
6
5
3
8
2
1
6
0
5
4
0
8
5
1
6
2
3
4
6
5
2
1
0
8
3
2
4
3
1
8
0
1
5
6
4
2
3
3
2
4
8
0
6
5
1
0
2
4
8
3
5
3
0
1
8
4
5
8
4
3
1
6
3
5
2
1
8
4
0
3
6
5
1
4
3
8
2
1
4
5
6
5
4
0
1
3
2
0
4
5
6
2
0
5
3
1
6
0
2
3
1
6
6
2
8
0
3
4
1
5
0
6
8
1
4
0
4
1
6
5
8
4
5
0
8
3
1
6
2
5
1
0
8
4
6
3
3
1
2
4
8
1
5
2
6
3
0
4
3
4
5
6
0
8
8
4
5
6
0
3
1
2
6
5
3
8
1
0
4
2
4
3
8
5
1
0
2
6
3
1
0
8
4
5
2
6
5
4
1
0
8
6
2
1
5
2
8
0
3
6
1
8
3
6
1
8
4
8
0
2
3
6
3
5
2
0
1
8
6
4
8
3
4
2
0
3
6
8
2
1
4
8
0
1
2
4
3
5
6
6
0
8
4
1
5
3
2
1
5
4
2
3
0
8
6
2
0
3
8
5
1
4
3
0
2
6
5
4
1
8
5
8
6
3
4
1
2
4
3
2
6
0
5
1
1
6
4
0
3
2
8
5
5
4
8
2
3
6
5
8
0
4
1
2
3
6
2
0
4
6
1
5
8
3
2
0
4
1
8
3
5
1
3
5
2
6
8
0
4
8
0
3
1
4
6
2
5
4
6
3
0
1
2
5
6
3
1
2
4
8
0
5
5
0
8
6
1
3
8
0
1
6
3
2
4
5
0
6
8
1
2
3
5
4
8
2
3
1
0
5
6
0
6
3
5
4
8
0
3
6
8
1
4
2


0
3
1
0
1
6
3
5
4
1
8
3
0
2
6
4
6
5
3
1
0
3
4
2
1
3
4
0
5
8
6
2
1
4
2
0
6
5
8
8
4
2
1
4
5
8
3
6
1
0
8
5
6
2
3
1
0
0
6
5
8
1
2
3
4
8
6
4
0
5
3
6
8
1
2
4
8
5
0
2
1
3
2
8
4
0
5
6
1
2
0
8
5
4
3
6
4
2
8
5
3
1
6
0
0
2
5
4
8
1
8
3
5
0
2
4
8
5
6
2
5
6
1
3
8
4
8
1
4
0
3
6
5
0
2
4
1
6
5
3
0
5
3
8
1
6
2
6
8
5
0
1
4
4
1
3
0
8
6
2
5
5
2
6
8
4
3
0
1
6
3
4
2
0
1
5
8
0
1
2
4
5
8
3
1
0
4
1
0
5
8
2
3
4
6
2
8
5
6
1
8
5
0
2
3
6
4
1
0
5
4
2
6
8
1
0
3
5
6
2
4
8
0
8
4
5
6
1
3
1
5
0
8
3
4
2
6
4
8
5
1
3
2
5
0
1
3
6
4
8
0
2
3
4
5
1
2
4
5
8
3
6
2
1
0
3
8
4
3
2
1
0
4
6
5
3
6
8
1
4
5
2
0
8
0
5
3
1
2
4
6
8
1
0
3
4
5
8
4
6
0
4
6
8
5
1
5
0
4
2
8
3
1
6
8
1
0
3
4
4
6
3
1
8
0
5
1
2
6
8
0
4
5
3
4
5
3
2
6
1
0
5
1
2
0
6
8
3
4
8
6
0
2
3
1
4
1
8
5
2
3
4
0
6
6
5
0
1
8
3
4
8
5
6
0
4
1
2
6
4
1
0
5
2
3
8
6
5
0
1
4
8
2
5
4
6
0
2
1
0
3
5
2
4
1
6
8
0
6
5
3
1
2
8
4
5
4
0
3
8
6
1
2
5
2
4
6
1
8
1
5
4
3
2
0
8
6
1
2
8
3
6
4
5
0
3
5
8
2
1
6
4
0
6
1
8
0
2
4
2
6
3
4
5
0
8
8
1
3
5
0
2
4
4
3
5
2
8
1
0
4
5
6
3
2
1
4
2
0
6
8
4
0
1
6
5
8
8
0
4


6
5
2
4
1
0
1
7
2
5
4
3
0
1
7
4
6
1
0
5
6
3
7
7
4
6
2
3
5
5
7
6
3
1
0
2
4
4
7
6
2
0
1
3
5
6
1
0
4
2
7
6
7
3
2
0
0
5
6
2
7
0
3
5
4
1
6
2
5
3
2
7
6
4
0
1
5
1
6
4
3
2
7
0
4
2
3


KeyboardInterrupt: 

Looks like the center and the corners are a lot better.

## Some Tests

In [14]:
# x is about to win (play 8)

board = empty_board() 
board[0] = 'x'
board[1] = 'o'
board[3] = 'o'
board[4] = 'x'

print("Board:")
show_board(board)

print()
%timeit -n1 -r1 display(pmcs(board))

Board:
[['x' 'o' ' ']
 ['o' 'x' ' ']
 [' ' ' ' ' ']]

Actions: [2, 5, 6, 7, 8] (20 playouts per actions)
5
6
6
5
7
8
5
7
6
8
7
5
8
6
6
8
8
5
7
6
5
6
7
8
8
6
7
6
6
5
7
8
7
6
8
7
5
6
6
8
5
6
6
8
5
7
8
6
8
5
6
7
7
6
7
6
7
2
8
6
2
8
8
6
7
2
2
8
7
6
2
8
8
7
6
2
8
6
7
2
2
8
6
7
2
8
7
2
8
6
8
2
7
6
8
7
2
6
2
7
8
6
8
6
2
7
6
7
8
2
2
7
6
8
7
2
8
6
2
7
8
6
7
6
2
8
6
8
2
5
8
7
8
5
7
2
2
8
2
5
8
7
2
8
7
2
5
2
2
7
8
5
2
5
8
7
2
8
8
7
2
5
7
2
2
8
5
8
5
2
5
2
7
2
8
5
7
2
7
8
2
7
8
5
8
6
5
2
2
5
6
8
2
6
5
8
2
5
6
8
2
5
8
6
6
2
5
8
8
2
6
5
8
2
6
5
2
5
8
6
6
2
8
5
2
8
6
8
8
6
2
5
2
5
6
8
8
6
2
5
5
2
8
6
6
8
5
6
2
8
8
6
5
2
5
8


{2: 0.95, 5: 0.7, 6: 0.7, 7: 0.65, 8: 1.0}

8

148 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
# o is about to win

board = empty_board() 
board[0] = 'o'
board[1] = 'o'
board[3] = 'o'
board[4] = 'x'
board[8] = 'x'

print("Board:")
show_board(board)

print()
%timeit -n1 -r1 display(pmcs(board))

print()
%timeit -n1 -r1 display(pmcs(board, N = 1000))

In [ ]:
#### x can draw if it chooses 7.

board = empty_board() 
board[0] = 'x'
board[1] = 'o'
board[2] = 'x'
board[4] = 'o'

print("Board:")
show_board(board)

print()
%timeit -n1 -r1 display(pmcs(board))

In [ ]:
# o went first

board = empty_board() 
board[4] = 'o'

print("Board:")
show_board(board)

print()
%timeit -n1 -r1 display(pmcs(board))

In [ ]:
# Empty board: Only a draw an be guaranteed

board = empty_board() 

print("Board:")
show_board(board)

print()
%timeit -n1 -r1 display(pmcs(board ))

In [ ]:
# A bad situation

board = empty_board() 
board[0] = 'o'
board[2] = 'x'
board[8] = 'o'

print("Board:")
show_board(board)

print()
%timeit -n1 -r1 display(pmcs(board))

__Note:__ It looks like random player o is very unlikely to block x and take advantage of the trap by playing the bottom left corner!

## Experiments


### Baseline: Randomized Player

A completely randomized player agent should be a weak baseline.

In [ ]:
def random_player(board, player = None):
    """Simple player that chooses a random empy square. player is unused"""
    return np.random.choice(get_actions(board))

show_board(board)
print()
%timeit -n1 -r1 random_player(board)

### The Environment

Implement the environment that calls the agent. The percept is the board and the action is move.

In [ ]:
DEBUG = 1

def switch_player(player, x, o):
    if player == 'x':
        return 'o', o
    else:
        return 'x', x

def play(x, o, N = 100):
    results = {'x': 0, 'o': 0, 'd': 0}
    for i in range(N):
        board = empty_board()
        player, fun = 'x', x
        
        while True:
            a = fun(board, player)
            board = result(board, player, a)
            if DEBUG >= 1: print(f"Player {player} uses action {a}")
            
            win = check_win(board)
            if win != 'n':
                if DEBUG >= 1: print(f"Result {board} winner: {win}")
                results[win] += 1
                break
            
            player, fun = switch_player(player, x, o)   
 
    return results

### Random vs. Random

In [ ]:
DEBUG = 0

%timeit -n 1 -r 1 display(play(random_player, random_player))

### Pure Monte Carlo Search vs. Random

In [ ]:
def pmcs10_player(board, player = 'x'):
    action = pmcs(board, N = 10, player = player)
    return action

def pmcs100_player(board, player = 'x'):
    action = pmcs(board, N = 100, player = player)
    return action

def pmcs1000_player(board, player = 'x'):
    action = pmcs(board, N = 1000, player = player)
    return action


DEBUG = 1
print("PMCS vs. random:")
display(play(pmcs10_player, random_player, N = 1))

In [ ]:
DEBUG = 0
print("PMCS vs. random:")
%timeit -n 1 -r 1 display(play(pmcs10_player, random_player))

print()
print("random vs. PMCS")
%timeit -n 1 -r 1 display(play(random_player, pmcs10_player))

In [ ]:
DEBUG = 0
print("PMCS vs. random:")
%timeit -n 1 -r 1 display(play(pmcs100_player, random_player))

print()
print("random vs. PMCS")
%timeit -n 1 -r 1 display(play(random_player, pmcs100_player))

In [ ]:
DEBUG = 0
print("PMCS (100) vs. PMCS (10):")
%timeit -n 1 -r 1 display(play(pmcs100_player, pmcs10_player))

print()
print("PMCS (10) vs. PMCS (100)")
%timeit -n 1 -r 1 display(play(pmcs10_player, pmcs100_player))

_Note 1:_ You can try `pmcs_100`, but it will take a few minutes to run 100 games.

_Note 2:_ The important advantage about Monte Carlo Search is that we do not need to define a heuristic evaluation function to decide what boards are good.